# PVLead data creation notebook 

In [69]:
import pandas as pd 
import xml.etree.ElementTree as ET

In [70]:
new_dir = "/Users/madsandersen/PycharmProjects/BscProjektData/BachelorProject/Data/PVELAD"
os.chdir(new_dir)

In [6]:
os.listdir()

['trainval', '.DS_Store', 'test', 'othertypes', 'annotation_classes.txt']

In [7]:
file_path = 'annotation_classes.txt'
with open(file_path, 'r') as file:
    folder_names = file.read().splitlines()

# Create folders based on the list of names
for folder_name in folder_names:
    folder_path = os.path.join(os.getcwd(), folder_name)  # Construct the full path
    os.makedirs(folder_path, exist_ok=True)  # Create the folder

print("Folders created successfully!")

Folders created successfully!


In [8]:
def extract_info_from_xml(xml_file):
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Extract filename
    filename = root.find('filename').text

    # Extract object name and bounding box
    object_elem = root.find('object')
    name = object_elem.find('name').text
    bndbox_elem = object_elem.find('bndbox')
    bndbox = {
        'xmin': int(bndbox_elem.find('xmin').text),
        'ymin': int(bndbox_elem.find('ymin').text),
        'xmax': int(bndbox_elem.find('xmax').text),
        'ymax': int(bndbox_elem.find('ymax').text)
    }

    return filename, name, bndbox

In [12]:
xml_file_path = "trainval/Annotations/img000001.xml"
extract_info_from_xml(xml_file_path)

('img000001.jpg',
 'black_core',
 {'xmin': 47, 'ymin': 69, 'xmax': 961, 'ymax': 967})

### Add the defects to a dataset 

In [53]:
directory = os.listdir('trainval/Annotations')
data = []
for xml_file in directory:
    root = 'trainval/Annotations/'
    filename,defect,bbox = extract_info_from_xml(root + xml_file)
    data.append({'file_name':filename,
                 'defect':defect,
                 'xmin': bbox['xmin'],
                 'ymin': bbox['ymin'],
                 'xmax': bbox['xmax'],
                 'ymax': bbox['ymax']})

### Add the non-defects to the data-set

In [47]:
directory = os.listdir('othertypes/good')
for img_file in directory:
    filename,defect,bbox = img_file,'Negative',{'xmin': None, 'ymin': None, 'xmax': None, 'ymax': None}
    data.append({'file_name':img_file,
                 'defect':defect,
                 'xmin': None,
                 'ymin': None,
                 'xmax': None,
                 'ymax': None})

In [54]:
df = pd.DataFrame(data)
df.to_csv('PVLEAD_dataset.csv')

In [51]:
def resize_images(input_folder, output_folder, target_size):
    # Loop through images in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # Open the image
            image = Image.open(image_path)

            # Resize the image
            resized_image = image.resize(target_size)

            # Save the resized image
            resized_image.save(output_path)

            #print(f"Resized image '{filename}' saved to '{output_folder}'.")

In [52]:
# Example usage:
input_folder_path = "trainval/JPEGImages"
output_folder_path = "PVLEAD_Defects/JPEGImages"
target_size = (320, 320)

resize_images(input_folder_path, output_folder_path, target_size)

In [67]:
len(df)

4500

##### im = Image.open('PVLEAD_Defects/JPEGImages/img001445.jpg')
(torchvision.transforms.ToTensor()(im))

## Making it compatiable with image folder

In [99]:
import ast

In [72]:
data = pd.read_csv("/Users/madsandersen/PycharmProjects/BscProjektData/BachelorProject/Data/VitusData/TrainVitusStat.csv")

In [112]:
def save_image_to_directory(image, destination_directory, filename):
    # Create the destination directory if it doesn't exist
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    # Build the destination file path
    destination_path = os.path.join(destination_directory, filename)

    # Save the image to the destination directory
    image.save(destination_path)
    #print(f"Saved {filename} to {destination_directory}")

In [113]:
positive_dir = '/Users/madsandersen/PycharmProjects/BscProjektData/BachelorProject/Data/VitusData/ImageFolderComp/Train/Positive'
negative_dir = '/Users/madsandersen/PycharmProjects/BscProjektData/BachelorProject/Data/VitusData/ImageFolderComp/Train/Negative'

for idx,row in data.iterrows():
    base_dir = '/Users/madsandersen/PycharmProjects/BscProjektData/BachelorProject/Data/'
    im_dir = base_dir + row['ImageDir']
    im = Image.open(im_dir)
    label = ast.literal_eval(row['Label'])
    file_name = row['ImageDir'].split('/')[-1]
    if 'Negative' == label[0]:
        destination_directory = negative_dir
        save_image_to_directory(im, destination_directory,file_name)
    else:
        destination_directory = positive_dir
        save_image_to_directory(im, destination_directory,file_name)


In [111]:
row['ImageDir'].split('/')[-1]

'Serie_4_ImageCorr_-9_4165_PC_Cell_Row9_Col_3.png'

In [302]:
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor,Resize
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, WeightedRandomSampler
import torchvision.models as models

In [323]:
t_forms = [
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    Resize((430,430)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ColorJitter(contrast=0.25,brightness = 0.25),
    transforms.RandomRotation(2),
    transforms.RandomRotation([-90,90]),
    transforms.RandomResizedCrop(size=(430, 430), scale=(0.7, 1.0)),
    transforms.Normalize((0.51, 0.51, 0.51), (0.15, 0.15, 0.15)) #padding_mode(132, 132, 132), (25, 25, 25)
]

transform_train = transforms.Compose(t_forms)

test_transform = transforms.Compose(
    [
     transforms.PILToTensor(),
     transforms.ConvertImageDtype(torch.float),
     Resize((430,430)),
     transforms.Normalize((0.51, 0.51, 0.51), (0.15, 0.15, 0.15))
    ]
)
train_set = torchvision.datasets.ImageFolder('/Users/madsandersen/PycharmProjects/BscProjektData/BachelorProject/Data/VitusData/ImageFolderComp/Train',
                                            transform=transform_train)
test_set = torchvision.datasets.ImageFolder('/Users/madsandersen/PycharmProjects/BscProjektData/BachelorProject/Data/VitusData/ImageFolderComp/Test',
                                            transform=test_transform)

In [324]:
targetlst = list(train_set.targets)
neg_weight,pos_weigth = 1/targetlst.count(0),1/targetlst.count(1)
random_sampler_weights = [neg_weight if x == 0 else pos_weigth for x in targetlst]
random_sampler_weights
sampler = WeightedRandomSampler(num_samples = len(train_set),
                                replacement = True,
                                weights = random_sampler_weights)

In [325]:
train_dataloader = DataLoader(train_set, batch_size=2,sampler =sampler )
test_dataloader = DataLoader(train_set, batch_size=2, shuffle=False)

In [306]:
next(iter(train_dataloader))[0].shape

torch.Size([32, 3, 430, 430])

### Training test

In [326]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [167]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [307]:
train_dataloader.targets

AttributeError: 'DataLoader' object has no attribute 'targets'

In [327]:
# Get cpu, gpu or mps device for training.
device = "cuda" if torch.cuda.is_available() else 'cpu'

print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(184900, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
model = models.mobilenet_v2(pretrained=False)
class MobileNetModules(nn.Module):
    def __init__(self, model):
        super(MobileNetModules, self).__init__()
        self.mobilenet = model
        #Input size is 440x440x1
        self.mobilenet.features[0][0] = nn.Conv2d(3, 32, kernel_size=7, stride=2, padding=3, bias=False)
        self.mobilenet.classifier[1] = nn.Linear(1280, 1)

    def forward(self, x):
        x = self.mobilenet(x)
        return x

model = MobileNetModules(model).to(device)   
print(model)

Using cpu device
MobileNetModules(
  (mobilenet): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2

In [328]:
loss_fn = nn.BCELoss()#nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [350]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        y.type(torch.int64)
        
        # Compute prediction error
        pred = model(X).flatten().type(torch.float64)
        print(pred.dtype)
        print(y.dtype)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [351]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            y.to(torch.int64)
            pred = model(X).type(torch.float32)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [352]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
torch.float64
torch.int64


RuntimeError: Found dtype Long but expected Double

In [301]:
next(iter(train_loader))

TypeError: randint() received an invalid combination of arguments - got (generator=list, dtype=torch.dtype, size=tuple, high=int, ), but expected one of:
 * (int high, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int high, tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int low, int high, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int low, int high, tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [207]:
x[torch.where(x == 3)]

tensor([3])

In [209]:
bla = x == 3

In [210]:
bla

tensor([False, False,  True, False])